In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Caricamento delle immagini in input in formato array numpy

In [2]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = '../Nuovi Dati/Dataset Classificatore'

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [3]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    for filename in os.listdir(path_cartella):
        if (filename.endswith(".jpg")):    
            path_file = f'{path_cartella}/{filename}'
            immagine = get_immagine_da_path(path_file)
            immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [25]:
image_list = []
label_list = []

cartelle = [('Buoni', 0), ('Polvere', 1)]

for cartella in cartelle:
    path_cartella = f'{IMAGE_PATH_FOLDER}/{cartella[0]}'
    label = cartella[1]
    immagini_grado = get_immagini_cartella(path_cartella)
    image_list.extend(immagini_grado)
    label_to_append = [label for x in range(len(immagini_grado))] 
    label_list.extend(label_to_append)
        
        

In [26]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(train_images, train_weights, test_size=0.20, random_state=42)

In [28]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output con attivazione sigmoide per la classificazione binaria
])

In [29]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [32]:
model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/10
257/257 [==============================] - 15s 59ms/step - loss: 0.1389 - accuracy: 0.9689
Epoch 2/10
257/257 [==============================] - 15s 59ms/step - loss: 0.1407 - accuracy: 0.9728
Epoch 3/10
257/257 [==============================] - 15s 59ms/step - loss: 0.6692 - accuracy: 0.9416
Epoch 4/10
257/257 [==============================] - 15s 59ms/step - loss: 4.3071 - accuracy: 0.9222
Epoch 5/10
257/257 [==============================] - 15s 59ms/step - loss: 0.1420 - accuracy: 0.9844
Epoch 6/10
257/257 [==============================] - 15s 59ms/step - loss: 0.0865 - accuracy: 0.9922
Epoch 7/10
257/257 [==============================] - 16s 61ms/step - loss: 0.0834 - accuracy: 0.9844
Epoch 8/10
257/257 [==============================] - 16s 64ms/step - loss: 0.0209 - accuracy: 0.9961
Epoch 9/10
257/257 [==============================] - 16s 62ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 10/10
257/257 [==============================] - 16s 61ms/step - loss: 9.734

In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

2023-07-26 15:45:11.453977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 114ms/step - loss: 0.6046 - accuracy: 0.9692
Test accuracy: 0.9692307710647583


In [ ]:
# predictions = model.predict(train_images)

In [33]:
# model.save('./modelli_classificatori/classificatore_black_stains')
# model.save('./modelli_classificatori/classificatore_blur')
model.save('./modelli_classificatori/classificatore_polvere')

INFO:tensorflow:Assets written to: ./modelli_classificatori/classificatore_polvere/assets


INFO:tensorflow:Assets written to: ./modelli_classificatori/classificatore_polvere/assets
